# Experiment 006: Fractional Translation (jonathanchan approach)

Apply micro-translations to the existing pre-optimized solution.
Steps: 0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001

This works on the existing optimized solution rather than starting from scratch.

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
import copy
import time

getcontext().prec = 25
scale_factor = Decimal("1e15")

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print("Setup complete")

In [ ]:
class ChristmasTree:
    """Christmas tree with Decimal precision"""
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        vertices = []
        for tx, ty in zip(TX, TY):
            vertices.append((float(Decimal(str(tx)) * scale_factor), 
                           float(Decimal(str(ty)) * scale_factor)))
        initial_polygon = Polygon(vertices)
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )
    
    def get_params(self):
        return self.center_x, self.center_y, self.angle
    
    def set_params(self, x, y, angle):
        self.center_x = Decimal(str(x))
        self.center_y = Decimal(str(y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

def has_overlap(trees, idx):
    """Check if tree at idx overlaps with any other tree"""
    for j, t2 in enumerate(trees):
        if idx != j:
            if trees[idx].polygon.intersects(t2.polygon) and not trees[idx].polygon.touches(t2.polygon):
                return True
    return False

def calculate_side(trees):
    """Calculate bounding box side length"""
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    return max(xys.max(axis=0) - xys.min(axis=0))

print("Classes defined")

In [ ]:
def fractional_translation(trees, max_iter=200):
    """
    Apply fractional translation optimization.
    From jonathanchan kernel.
    """
    best_trees = [t.clone() for t in trees]
    best_side = calculate_side(best_trees)
    
    frac_steps = [0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001]
    dx_list = [0, 0, 1, -1, 1, 1, -1, -1]
    dy_list = [1, -1, 0, 0, 1, -1, 1, -1]
    
    improvements = 0
    
    for iteration in range(max_iter):
        improved = False
        
        for i in range(len(best_trees)):
            for step in frac_steps:
                for d in range(8):
                    ox, oy, oa = best_trees[i].get_params()
                    
                    new_x = ox + Decimal(str(dx_list[d] * step))
                    new_y = oy + Decimal(str(dy_list[d] * step))
                    
                    best_trees[i].set_params(new_x, new_y, oa)
                    
                    if not has_overlap(best_trees, i):
                        new_side = calculate_side(best_trees)
                        if new_side < best_side - 1e-12:
                            best_side = new_side
                            improved = True
                            improvements += 1
                        else:
                            best_trees[i].set_params(ox, oy, oa)
                    else:
                        best_trees[i].set_params(ox, oy, oa)
        
        if not improved:
            break
        
        if iteration % 10 == 0:
            print(f"  Iter {iteration}: side={best_side:.9f}, improvements={improvements}")
    
    return best_trees, best_side

print("Fractional translation defined")

In [ ]:
# Load baseline submission
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

print("Loaded baseline")

In [ ]:
# Test fractional translation on a few N values
test_ns = [10, 20, 50, 100]

results = {}
for n in test_ns:
    print(f"\n=== N={n} ===")
    trees = load_trees_for_n(baseline_df, n)
    original_side = calculate_side(trees)
    original_score = original_side**2 / n
    print(f"Original: side={original_side:.9f}, score={original_score:.9f}")
    
    t0 = time.time()
    optimized_trees, new_side = fractional_translation(trees, max_iter=50)
    elapsed = time.time() - t0
    
    new_score = new_side**2 / n
    improvement = original_score - new_score
    
    print(f"Optimized: side={new_side:.9f}, score={new_score:.9f}")
    print(f"Improvement: {improvement:.12f} ({improvement/original_score*100:.6f}%)")
    print(f"Time: {elapsed:.1f}s")
    
    results[n] = {
        'original_score': original_score,
        'new_score': new_score,
        'improvement': improvement,
        'trees': optimized_trees
    }

In [ ]:
# Summary
print("\n" + "="*50)
print("FRACTIONAL TRANSLATION SUMMARY")
print("="*50)

total_improvement = 0
for n, r in results.items():
    print(f"N={n}: {r['original_score']:.9f} -> {r['new_score']:.9f} (improvement: {r['improvement']:.12f})")
    total_improvement += r['improvement']

print(f"\nTotal improvement on tested N values: {total_improvement:.12f}")

if total_improvement > 0:
    print("\nFractional translation found improvements!")
else:
    print("\nNo improvements found - baseline is already at local optimum.")